# Conclusion :

A l'état brut, les différentes formes de SVM n'ont pas réussi à faire mieux que la régression logistique
Dans ce cas, on va donc maximiser la régression logistique. Mais avant celà, il faut qu'on établisse une fonction pour entrainer automatiquement les modèles et sortir plus de métriques intéressantes

In [1]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
from evaluator import *
from sklearn.svm import SVC

In [8]:
df_train = pd.read_csv('df_train.csv')
df_validation = pd.read_csv('df_validation.csv')

# Normalisation

In [9]:
X_train = df_train.iloc[:, :768]
Y_train = df_train.iloc[:, 768]
gender_train = df_train.iloc[:, 769]

In [12]:
X_validation = df_validation.iloc[:, :768]
Y_validation = df_validation.iloc[:, 768]
gender_validation = df_validation.iloc[:, 769]

In [10]:
# Création de l'instance scaler
scaler = StandardScaler()

# Ajustement du scaler sur le jeu de données d'entraînement
scaler.fit(X_train)

# Normalisation de X_train et X_validation
X_train_normalized = scaler.transform(X_train)
X_validation_normalized = scaler.transform(X_validation)

# Différents SVM sans normalisation 

# Différents SVM sans normalisation

In [ ]:
def evaluate_model(model, X_train, Y_train, X_validation, Y_validation, gender_validation):
    # Entraîner le modèle sur les données d'entraînement
    model.fit(X_train, Y_train)

    # Faire des prédictions sur l'ensemble de validation
    predictions = model.predict(X_validation)

    # Évaluer les scores, en utilisant une fonction gap_eval_scores
    eval_scores, confusion_matrices_eval = gap_eval_scores(predictions, Y_validation, gender_validation, metrics=['TPR', 'FPR', 'PPR'])

    # Calculer le final_score
    final_score = (eval_scores['macro_fscore'] + (1 - eval_scores['TPR_GAP'])) / 2
    
    return final_score

In [14]:
from sklearn.svm import LinearSVC, SVC

# Configuration et évaluation des différents modèles SVM avec des ajustements pour la convergence
models = [
    LinearSVC(max_iter=10000, dual=False, tol=1e-4),  # Ajustement de 'max_iter', 'dual', et 'tol' pour LinearSVC
    SVC(kernel='linear', tol=1e-4),  # Ajustement de 'tol' pour SVC avec kernel linéaire
    SVC(kernel='poly', degree=3, tol=1e-4),  # Ajustement de 'tol' pour SVC avec kernel polynomial
    SVC(kernel='rbf', tol=1e-4),  # Ajustement de 'tol' pour SVC avec kernel RBF
    SVC(kernel='sigmoid', tol=1e-4)  # Ajustement de 'tol' pour SVC avec kernel sigmoid
]

for model in models:
    try:
        final_score = evaluate_model(model, X_train, Y_train, X_validation, Y_validation, gender_validation)
        print(f"Modèle {model.__class__.__name__} avec kernel={getattr(model, 'kernel', 'N/A')}, Score Final: {final_score}")
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle {model.__class__.__name__} avec kernel={getattr(model, 'kernel', 'N/A')}: {e}")

Modèle LinearSVC avec kernel=N/A, Score Final: 0.7188068032699206
Modèle SVC avec kernel=linear, Score Final: 0.7221988908233821
Modèle SVC avec kernel=poly, Score Final: 0.7164003376991557
Modèle SVC avec kernel=rbf, Score Final: 0.7054093884407573
Modèle SVC avec kernel=sigmoid, Score Final: 0.6743839171110846


# Différents SVM avec normalisation

In [24]:
from sklearn.svm import LinearSVC, SVC

# Configuration et évaluation des différents modèles SVM avec des ajustements pour la convergence
models = [
    LinearSVC(max_iter=10000, dual=False, tol=1e-4),  # Ajustement de 'max_iter', 'dual', et 'tol' pour LinearSVC
    SVC(kernel='linear', tol=1e-4),  # Ajustement de 'tol' pour SVC avec kernel linéaire
    SVC(kernel='poly', degree=3, tol=1e-4),  # Ajustement de 'tol' pour SVC avec kernel polynomial
    SVC(kernel='rbf', tol=1e-4),  # Ajustement de 'tol' pour SVC avec kernel RBF
    SVC(kernel='sigmoid', tol=1e-4)  # Ajustement de 'tol' pour SVC avec kernel sigmoid
]

for model in models:
    try:
        final_score = evaluate_model(model, X_train_normalized, Y_train, X_validation_normalized, Y_validation, gender_validation)
        print(f"Modèle {model.__class__.__name__} avec kernel={getattr(model, 'kernel', 'N/A')}, Score Final: {final_score}")
    except Exception as e:
        print(f"Erreur lors de l'évaluation du modèle {model.__class__.__name__} avec kernel={getattr(model, 'kernel', 'N/A')}: {e}")

Modèle LinearSVC avec kernel=N/A, Score Final: 0.7034994213644952
Modèle SVC avec kernel=linear, Score Final: 0.7125674474041342
Modèle SVC avec kernel=poly, Score Final: 0.654142888705759
Modèle SVC avec kernel=rbf, Score Final: 0.7242901352024325
Modèle SVC avec kernel=sigmoid, Score Final: 0.7071213092822344
